In [8]:
import os
import h3
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd

from pyproj import CRS, Transformer
from shapely.geometry import Polygon
import contextily as ctx


In [9]:
# Create directory for frames
OUTPUT_DIR = "h3_animation_frames"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

df = pd.read_csv("coordinates.csv", index_col=False)


lat_min, lat_max = df.agg(["min", "max"])["lat"]
lon_min, lon_max = df.agg(["min", "max"])["lon"]


# Define an Istanbul bounding box (lon_min, lat_min, lon_max, lat_max)
bbox = (28.6, 40.85, 29.3, 41.25)

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["lon"], df["lat"]),
    crs="EPSG:4326"
).to_crs(epsg=3857)


vec_latlng_to_cell = np.vectorize(lambda lat, lng: h3.latlng_to_cell(lat, lng, 8))
vec_h3_to_geo = np.vectorize(h3.cell_to_latlng)

lat = gdf["lat"].to_numpy()
lon = gdf["lon"].to_numpy()

h3_indexes = vec_latlng_to_cell(lat, lon)
h3_lat, h3_lon = vec_h3_to_geo(h3_indexes)


# Transformer from WGS84 (4326) to Web Mercator (3857)
transformer = Transformer.from_crs(CRS.from_user_input(4326), CRS.from_user_input(3857), always_xy=True)

polys = [
    Polygon([(lon, lat) for lat, lon in h3.cell_to_boundary(h)])
    for h in h3_indexes]

hex_gdf = gpd.GeoDataFrame(
    {"h3_index": h3_indexes, "h3_lat": h3_lat, "h3_lon": h3_lon}, geometry=polys, crs="EPSG:4326").to_crs(epsg=3857)

# Apply the transformation to the h3_lon and h3_lat columns
hex_gdf["h3_center_x"], hex_gdf["h3_center_y"] = transformer.transform(
    hex_gdf["h3_lon"].to_numpy(), 
    hex_gdf["h3_lat"].to_numpy()
)

minx, miny, maxx, maxy = gdf.total_bounds

# Add a small padding around the data
pad_x = (maxx - minx) * 0.2
pad_y = (maxy - miny) * 0.2


for i in range(0, len(hex_gdf) + 1):
    # Plot
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_axis_off()

    # Set limits before any plotting
    ax.set_xlim(minx - pad_x, maxx + pad_x)
    ax.set_ylim(miny - pad_y, maxy + pad_y)
    
    sns.scatterplot(
    data=gdf,
    x=gdf.geometry.x[i:],
    y=gdf.geometry.y[i:],
    ax=ax,
    s=45
    )
    
    hex_gdf.iloc[0:i].plot(ax=ax, alpha=0.3, edgecolor="black")
    ax.plot(hex_gdf["h3_center_x"].iloc[0:i], hex_gdf["h3_center_y"].iloc[0:i], "o", color="red",
             markeredgecolor="white", markersize=15)


    # Add background
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
    plt.savefig(f"{OUTPUT_DIR}/frame_{i:03d}.png", bbox_inches='tight', pad_inches=0, dpi=200)
    plt.close(fig)


C:\Users\abduy\AppData\Local\Temp\ipykernel_5064\3906231892.py:73: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  hex_gdf.iloc[0:i].plot(ax=ax, alpha=0.3, edgecolor="black")
